In [1]:
import numpy as np
import pandas as pd
import json
import os
import re

In [2]:
# get all json file path in the folder
folder_path = "../dataset/USA"

file_paths = []
for filename in os.listdir(folder_path):
    filepath = os.path.join(folder_path, filename)
    if os.path.isfile(filepath):
        file_paths.append(filepath)

len(file_paths)

47

In [3]:
job_results_by_types = {}

for file_path in file_paths:
    match = re.search(r'/([^/]+)-\d+\.json$', file_path)
    if match:
        job_type = match.group(1)
        with open(file_path, 'r') as f:
            data = json.load(f)
        job_results_by_types[job_type] = job_results_by_types.get(job_type, []) + [data,]

print(job_results_by_types.keys())

dict_keys(['block-chain', 'data-scientist', 'natural-language-processing', 'reinforcement-learning', 'neural-networks', 'big-data-and-cloud-computing', 'data-analyst', 'machine-learning', 'time-series-analysis', 'deep-learning'])


In [4]:
jobs_data = pd.DataFrame(columns=["type",'title', 'company_name', 'location', 'via', 'description', 'job_id'])

In [7]:
for job_type in job_results_by_types:
    datasets = job_results_by_types[job_type]
    for dataset in datasets:
        if 'jobs_results' not in dataset:
            continue 
        jobs_results = dataset["jobs_results"]
        for job_result in jobs_results:
            title = job_result["title"]
            company_name = job_result["company_name"]
            location = job_result["location"] 
            via = job_result["via"] 
            description = job_result["description"] 

            job_id = job_result["job_id"]
            data = {'type': job_type, 'title': title, 'company_name': company_name, 'location': location, 'via': via, 'description': description,'job_id': job_id}
            jobs_data = jobs_data.append(data, ignore_index=True)
jobs_data.head()

/var/folders/h0/sq_brb3d0_lfvtk4fhlbrck00000gn/T/ipykernel_46091/1718973618.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  jobs_data = jobs_data.append(data, ignore_index=True)
/var/folders/h0/sq_brb3d0_lfvtk4fhlbrck00000gn/T/ipykernel_46091/1718973618.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  jobs_data = jobs_data.append(data, ignore_index=True)
/var/folders/h0/sq_brb3d0_lfvtk4fhlbrck00000gn/T/ipykernel_46091/1718973618.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  jobs_data = jobs_data.append(data, ignore_index=True)
/var/folders/h0/sq_brb3d0_lfvtk4fhlbrck00000gn/T/ipykernel_46091/1718973618.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future vers

,type,title,company_name,location,via,description,job_id
0,block-chain,Senior Blockchain Software Engineer,Truist,Anywhere,via ZipRecruiter,The position is described below. If you want t...,eyJqb2JfdGl0bGUiOiJTZW5pb3IgQmxvY2tjaGFpbiBTb2...
1,block-chain,Get twitter accounts that are tied to blockcha...,Upwork,Anywhere,via Upwork,I am looking for a list of blockchain addresse...,eyJqb2JfdGl0bGUiOiJHZXQgdHdpdHRlciBhY2NvdW50cy...
2,block-chain,Blockchain Analyst,Allium,"New York, NY",via Jobs,About the role\n• Sherlock & Enola Holmes leve...,eyJqb2JfdGl0bGUiOiJCbG9ja2NoYWluIEFuYWx5c3QiLC...
3,block-chain,Full Stack Blockchain Engineer,geojam,"Los Angeles, CA",via AngelList,"Who We Are\n\nLaunched in 2020, Geojam is a so...",eyJqb2JfdGl0bGUiOiJGdWxsIFN0YWNrIEJsb2NrY2hhaW...
4,block-chain,Blockchain Engineer - Apps Platform at Exodus ...,Exodus,"Livermore, CA","via Livermore, CA - Geebo",Exodus is looking for a detail-orientated Bloc...,eyJqb2JfdGl0bGUiOiJCbG9ja2NoYWluIEVuZ2luZWVyIC...


In [8]:
subset_dataset = jobs_data.dropna(subset=['location'])
subset_dataset = subset_dataset[~subset_dataset['location'].str.contains('Anywhere|United States')]
pattern = r"\s*\([^)]*\)"
subset_dataset["location"] = subset_dataset["location"].str.replace(pattern, "", regex=True)
subset_dataset['via'] = subset_dataset['via'].str.replace('via ', '')
print(subset_dataset['location'].value_counts())

  New York, NY                 24
  San Francisco, CA            16
  Annapolis Junction, MD       12
  Chicago, IL                  11
  Palo Alto, CA                 9
                               ..
  Berkeley, CA                  1
  Milwaukee, WI                 1
  North Reading, MA             1
  Waltham, MA                   1
  Lincolnshire, IL              1
Name: location, Length: 135, dtype: int64


In [9]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

def geocode_location(location):
    try:
        geolocator = Nominatim(user_agent="my-custom-agent")
        location = geolocator.geocode(location)
        if location is None:
            return None
        else:
            return (location.latitude, location.longitude)
    except GeocoderTimedOut:
        return None

subset_dataset['location_coords'] = subset_dataset['location'].apply(geocode_location)
subset_dataset[['latitude', 'longitude']] = pd.DataFrame(subset_dataset['location_coords'].tolist(), index=subset_dataset.index)

In [19]:
subset_dataset.head()

,type,title,company_name,location,via,description,job_id,location_coords,latitude,longitude
2,block-chain,Blockchain Analyst,Allium,"New York, NY",Jobs,About the role\n• Sherlock & Enola Holmes leve...,eyJqb2JfdGl0bGUiOiJCbG9ja2NoYWluIEFuYWx5c3QiLC...,"(40.7127281, -74.0060152)",40.712728,-74.006015
3,block-chain,Full Stack Blockchain Engineer,geojam,"Los Angeles, CA",AngelList,"Who We Are\n\nLaunched in 2020, Geojam is a so...",eyJqb2JfdGl0bGUiOiJGdWxsIFN0YWNrIEJsb2NrY2hhaW...,"(34.0536909, -118.242766)",34.053691,-118.242766
4,block-chain,Blockchain Engineer - Apps Platform at Exodus ...,Exodus,"Livermore, CA","Livermore, CA - Geebo",Exodus is looking for a detail-orientated Bloc...,eyJqb2JfdGl0bGUiOiJCbG9ja2NoYWluIEVuZ2luZWVyIC...,"(37.6820583, -121.768053)",37.682058,-121.768053
5,block-chain,Blockchain Data Analytics: Tech Lead - Assista...,EY,"New York, NY",EY Careers,"At EY, you’ll have the chance to build a caree...",eyJqb2JfdGl0bGUiOiJCbG9ja2NoYWluIERhdGEgQW5hbH...,"(40.7127281, -74.0060152)",40.712728,-74.006015
6,block-chain,STI25526 Technical Architect Block Chain,Software Technology Inc.,"Irving, TX",LinkedIn,Title: Technical Architect Block Chain\n\nLoca...,eyJqb2JfdGl0bGUiOiJTVEkyNTUyNiBUZWNobmljYWwgQX...,"(32.8295183, -96.9442177)",32.829518,-96.944218


In [16]:
subset_dataset.to_csv('../dataset/USA_dataset_title.csv', index=False)

In [67]:
# get all json file path in the folder
folder_path = "../dataset/DC"

file_paths = []
for filename in os.listdir(folder_path):
    filepath = os.path.join(folder_path, filename)
    if os.path.isfile(filepath):
        file_paths.append(filepath)

len(file_paths)
job_results_by_types = {}

for file_path in file_paths:
    match = re.search(r'/([^/]+)-\d+\.json$', file_path)
    if match:
        job_type = match.group(1)
        with open(file_path, 'r') as f:
            data = json.load(f)
        job_results_by_types[job_type] = job_results_by_types.get(job_type, []) + [data,]
jobs_data2 = pd.DataFrame(columns=["type",'title', 'company_name', 'location', 'via', 'description', 'job_id'])
for job_type in job_results_by_types:
    datasets = job_results_by_types[job_type]
    for dataset in datasets:
        if 'jobs_results' not in dataset:
            continue 
        jobs_results = dataset["jobs_results"]
        for job_result in jobs_results:
            title = job_result["title"]
            company_name = job_result["company_name"]
            location = job_result["location"] 
            via = job_result["via"] 
            description = job_result["description"] 

            job_id = job_result["job_id"]
            data = {'type': job_type, 'title': title, 'company_name': company_name, 'location': location, 'via': via, 'description': description,'job_id': job_id}
            jobs_data2 = jobs_data2.append(data, ignore_index=True)
jobs_data2.head()
jobs_data2 = jobs_data2.dropna(subset=['location'])
jobs_data2 = jobs_data2[~jobs_data2['location'].str.contains('Anywhere|United States')]
pattern = r"\s*\([^)]*\)"
jobs_data2["location"] = jobs_data2["location"].str.replace(pattern, "", regex=True)
jobs_data2['via'] = jobs_data2['via'].str.replace('via ', '')
print(jobs_data2['location'].value_counts())


  Washington, DC          40
  New York, NY            21
  Arlington, VA           18
  Herndon, VA             13
  San Francisco, CA       12
                          ..
  College Park, MD         1
  Sacramento, CA           1
  Culver City, CA          1
  Cincinnati, OH           1
  Sterling, VA             1
Name: location, Length: 84, dtype: int64


/var/folders/h0/sq_brb3d0_lfvtk4fhlbrck00000gn/T/ipykernel_46091/2488248811.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  jobs_data2 = jobs_data2.append(data, ignore_index=True)
/var/folders/h0/sq_brb3d0_lfvtk4fhlbrck00000gn/T/ipykernel_46091/2488248811.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  jobs_data2 = jobs_data2.append(data, ignore_index=True)
/var/folders/h0/sq_brb3d0_lfvtk4fhlbrck00000gn/T/ipykernel_46091/2488248811.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  jobs_data2 = jobs_data2.append(data, ignore_index=True)
/var/folders/h0/sq_brb3d0_lfvtk4fhlbrck00000gn/T/ipykernel_46091/2488248811.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a futur

In [70]:
jobs_data2.head()

,type,title,company_name,location,via,description,job_id
1,block-chain,Blockchain Engineer,21.co,"New York, NY",Greenhouse,We are seeking a highly motivated and skilled ...,eyJqb2JfdGl0bGUiOiJCbG9ja2NoYWluIEVuZ2luZWVyIi...
10,block-chain,Blockchain & Digital Assets Senior Manager,Deloitte,"Arlington, VA",Deloitte Jobs,We are expanding Deloitte's Audit & Assurance ...,eyJqb2JfdGl0bGUiOiJCbG9ja2NoYWluIFx1MDAyNiBEaW...
11,block-chain,Blockchain Technical Expert,Swift,"Manassas, VA",LinkedIn,About The Role\n\nBlockchain Technical Expert....,eyJqb2JfdGl0bGUiOiJCbG9ja2NoYWluIFRlY2huaWNhbC...
12,block-chain,Blockchain Architect *US Citizenship Required*,"CGI Group, Inc.","Fairfax, VA",WJHL Jobs,Position Description:\n\nCGI Federal is lookin...,eyJqb2JfdGl0bGUiOiJCbG9ja2NoYWluIEFyY2hpdGVjdC...
13,block-chain,Developer Relations Manager at Flow Blockchain...,Flow Blockchain,"Rockville, MD","Rockville, MD - Geebo",We're looking for an experienced Developer Rel...,eyJqb2JfdGl0bGUiOiJEZXZlbG9wZXIgUmVsYXRpb25zIE...


In [71]:

from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

def geocode_location(location):
    try:
        geolocator = Nominatim(user_agent="my-custom-agent")
        location = geolocator.geocode(location)
        if location is None:
            return None
        else:
            return (location.latitude, location.longitude)
    except GeocoderTimedOut:
        return None

jobs_data2['location_coords'] = jobs_data2['location'].apply(geocode_location)
jobs_data2[['latitude', 'longitude']] = pd.DataFrame(jobs_data2['location_coords'].tolist(), index=jobs_data2.index)


In [72]:
combined_dataset = pd.concat([subset_dataset, jobs_data2], ignore_index=True)

combined_dataset.head()

,type,title,company_name,location,via,description,job_id,location_coords,latitude,longitude
0,block-chain,Blockchain Analyst,Allium,"New York, NY",Jobs,About the role\n• Sherlock & Enola Holmes leve...,eyJqb2JfdGl0bGUiOiJCbG9ja2NoYWluIEFuYWx5c3QiLC...,"(40.7127281, -74.0060152)",40.712728,-74.006015
1,block-chain,Full Stack Blockchain Engineer,geojam,"Los Angeles, CA",AngelList,"Who We Are\n\nLaunched in 2020, Geojam is a so...",eyJqb2JfdGl0bGUiOiJGdWxsIFN0YWNrIEJsb2NrY2hhaW...,"(34.0536909, -118.242766)",34.053691,-118.242766
2,block-chain,Blockchain Engineer - Apps Platform at Exodus ...,Exodus,"Livermore, CA","Livermore, CA - Geebo",Exodus is looking for a detail-orientated Bloc...,eyJqb2JfdGl0bGUiOiJCbG9ja2NoYWluIEVuZ2luZWVyIC...,"(37.6820583, -121.768053)",37.682058,-121.768053
3,block-chain,Blockchain Data Analytics: Tech Lead - Assista...,EY,"New York, NY",EY Careers,"At EY, you’ll have the chance to build a caree...",eyJqb2JfdGl0bGUiOiJCbG9ja2NoYWluIERhdGEgQW5hbH...,"(40.7127281, -74.0060152)",40.712728,-74.006015
4,block-chain,STI25526 Technical Architect Block Chain,Software Technology Inc.,"Irving, TX",LinkedIn,Title: Technical Architect Block Chain\n\nLoca...,eyJqb2JfdGl0bGUiOiJTVEkyNTUyNiBUZWNobmljYWwgQX...,"(32.8295183, -96.9442177)",32.829518,-96.944218


In [73]:
combined_dataset.to_csv('../dataset/combined_dataset_title.csv', index=False)

In [74]:
import folium

loc = 'Job Information Across the United States'
title_html = '''
             <h3 align="center" style="font-size:20px"><b>{}</b></h3>
             '''.format(loc)  
# Create a map centered at the region's coordinates
map_obj = folium.Map(location=[37.0902, -95.7129], zoom_start=4)

# Add a marker for each job location
for index, row in combined_dataset.iterrows():
    job_coords = (row['latitude'], row['longitude'])
    job_title = row['title']
    job_company = row['company_name']
    job_type = row['type']
    via = row['via']


    popup_text = f"{job_company}: {job_title}({via})"
    if job_type == 'block-chain':
        icon_color = 'red'
    elif job_type == 'data-scientist':
        icon_color = 'blue'
    elif job_type == 'natural-language-processing':
        icon_color = 'orange'
    elif job_type == 'reinforcement-learning':
        icon_color = 'green'
    elif job_type == 'neural-networks':
        icon_color = 'purple'
    elif job_type == 'big-data-and-cloud-computing':
        icon_color = 'cadetblue'
    elif job_type == 'data-analyst':
        icon_color = 'pink'
    elif job_type == 'machine-learning':
        icon_color = 'beige'
    elif job_type == 'time-series-analysis':
        icon_color = 'lightgreen'
    elif job_type == 'deep-learning':
        icon_color = 'black'
    icon = folium.Icon(color=icon_color, icon='briefcase', prefix='fa')
    marker = folium.Marker(location=job_coords, popup=popup_text, icon=icon)
    marker.add_to(map_obj)
# Add Category Legend
legend_html = """
<div style="position:fixed;
     bottom: 30px; 
     left: 20px; 
     width: 140px; 
     height: 250px; 
     border:2px solid grey; 
     z-index: 9999;
     font-size:12px;">
     &nbsp;<b>Job Types:</b><br>
     &nbsp;<i class="fa fa-circle fa-1x" style="color:red"></i>&nbsp;<b>block-chain</b><br>
     &nbsp;<i class="fa fa-circle fa-1x" style="color:blue"></i>&nbsp;<b>data-scientist</b><br>
     &nbsp;<i class="fa fa-circle fa-1x" style="color:orange"></i>&nbsp;<b>natural-language-processing</b><br>
     &nbsp;<i class="fa fa-circle fa-1x" style="color:green"></i>&nbsp;<b>reinforcement-learning</b><br>
     &nbsp;<i class="fa fa-circle fa-1x" style="color:purple"></i>&nbsp;<b>neural-networks</b><br>
     &nbsp;<i class="fa fa-circle fa-1x" style="color:cadetblue"></i>&nbsp;<b>big-data-and-cloud-computing</b><br>
     &nbsp;<i class="fa fa-circle fa-1x" style="color:pink"></i>&nbsp;<b>data-analyst</b><br>
    &nbsp;<i class="fa fa-circle fa-1x" style="color:beige"></i>&nbsp;<b>machine-learning</b><br>
     &nbsp;<i class="fa fa-circle fa-1x" style="color:lightgreen"></i>&nbsp;<b>time-series-analysis</b><br>
     &nbsp;<i class="fa fa-circle fa-1x" style="color:black"></i>&nbsp;<b>deep-learning</b><br>
               
</div>"""
map_obj.get_root().html.add_child(folium.Element(legend_html))
map_obj.get_root().html.add_child(folium.Element(title_html))


# Display the map
map_obj


In [75]:
map_obj.save("../images/map.html")